# Collaborative Filtering for Movie Recommendations

This example demonstrates Collaborative filtering using the Movielens dataset to recommend movies to users. The MovieLens ratings dataset lists the ratings given by a set of users to a set of movies. Our goal is to be able to predict ratings for movies a user has not yet watched. The movies with the highest predicted ratings can then be recommended to the user.

The steps in the model are as follows:



1.   Map user ID to a "user vector" via an embedding matrix
2. Map movie ID to a "movie vector" via an embedding matrix
3. Compute the dot product between the user vector and movie vector, to obtain the a match score between the user and the movie (predicted rating).
4. Train the embeddings via gradient descent using all known user-movie pairs.



In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

# First, load the data and apply preprocessing

In [2]:
# Download the actual data from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# Use the ratings.csv file
movielens_data_file_url = (
    "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
)
movielens_zipped_file = keras.utils.get_file(
    "ml-latest-small.zip", movielens_data_file_url, extract=False
)
keras_datasets_path = Path(movielens_zipped_file).parents[0]
movielens_dir = keras_datasets_path / "ml-latest-small"

# Only extract the data the first time the script is run.
if not movielens_dir.exists():
    with ZipFile(movielens_zipped_file, "r") as zip:
        # Extract files
        print("Extracting all the files now...")
        zip.extractall(path=keras_datasets_path)
        print("Done!")

ratings_file = movielens_dir / "ratings.csv"
df = pd.read_csv(ratings_file)

978202/978202 [==============================] - 0s 0us/step
Extracting all the files now...
Done!


First, need to perform some preprocessing to encode users and movies as integer indices.



In [3]:
user_ids = df["userId"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = df["movieId"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
df["user"] = df["userId"].map(user2user_encoded)
df["movie"] = df["movieId"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
df["rating"] = df["rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(df["rating"])
max_rating = max(df["rating"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

Number of users: 610, Number of Movies: 9724, Min rating: 0.5, Max rating: 5.0


# Prepare training and validation data

In [4]:
df = df.sample(frac=1, random_state=42)
x = df[["user", "movie"]].values
# Normalize the targets between 0 and 1. Makes it easy to train.
y = df["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

# Create the model


We embed both users and movies in to 50-dimensional vectors.

The model computes a match score between user and movie embeddings via a dot product, and adds a per-movie and per-user bias. The match score is scaled to the `[0, 1]` interval via a sigmoid (since our ratings are normalized to this range).

In [5]:
EMBEDDING_SIZE = 50


class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

# Train the model based on the data split

Setting early stopping condition to prevent overfitting

In [9]:
callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto'
)

In [10]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[callback]
)

Epoch 1/10
1418/1418 [==============================] - 14s 10ms/step - loss: 0.6064 - val_loss: 0.6144
Epoch 2/10
1418/1418 [==============================] - 14s 10ms/step - loss: 0.6073 - val_loss: 0.6142
Epoch 3/10
1418/1418 [==============================] - 13s 9ms/step - loss: 0.6056 - val_loss: 0.6136
Epoch 4/10
1418/1418 [==============================] - 14s 10ms/step - loss: 0.6049 - val_loss: 0.6137


As we can see, we stop after 4 epochs as validation loss starts to rise.

# Testing other batch sizes

Manually finding the loss of different batch sizes on the model

In [12]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[callback]
)

Epoch 1/10
2836/2836 [==============================] - 28s 10ms/step - loss: 0.5987 - val_loss: 0.6100
Epoch 2/10
2836/2836 [==============================] - 28s 10ms/step - loss: 0.5976 - val_loss: 0.6103


In [13]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[callback]
)

Epoch 1/10
709/709 [==============================] - 7s 10ms/step - loss: 0.6284 - val_loss: 0.6584
Epoch 2/10
709/709 [==============================] - 8s 12ms/step - loss: 0.6386 - val_loss: 0.6378
Epoch 3/10
709/709 [==============================] - 8s 11ms/step - loss: 0.6398 - val_loss: 0.6438


In [14]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[callback]
)

Epoch 1/10
355/355 [==============================] - 3s 9ms/step - loss: 0.6919 - val_loss: 0.6545
Epoch 2/10
355/355 [==============================] - 4s 12ms/step - loss: 0.6897 - val_loss: 0.6518
Epoch 3/10
355/355 [==============================] - 3s 9ms/step - loss: 0.7012 - val_loss: 0.6539


Our best results were from lower batch sizes. As the batch size increased, both our validation loss and standard loss increased. In fact, with batch sizes with higher than 64, our loss increased showing our fit was getting worse on our data as we trained more.

# Manually testing different embedding sizes

Testing various embedding sizes effect on our best batch size model's loss

In [22]:
EMBEDDING_SIZE = 100
model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

In [24]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[callback]
)

Epoch 1/10
2836/2836 [==============================] - 46s 16ms/step - loss: 0.6300 - val_loss: 0.6200
Epoch 2/10
2836/2836 [==============================] - 43s 15ms/step - loss: 0.6171 - val_loss: 0.6209


In [25]:
EMBEDDING_SIZE = 150
model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

In [26]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[callback]
)

Epoch 1/10
2836/2836 [==============================] - 68s 23ms/step - loss: 0.6324 - val_loss: 0.6227
Epoch 2/10
2836/2836 [==============================] - 67s 24ms/step - loss: 0.6295 - val_loss: 0.6310


In [27]:
EMBEDDING_SIZE = 200
model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

In [28]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks=[callback]
)

Epoch 1/10
2836/2836 [==============================] - 86s 30ms/step - loss: 0.6335 - val_loss: 0.6241
Epoch 2/10
2836/2836 [==============================] - 84s 30ms/step - loss: 0.6466 - val_loss: 0.6438


At the batch size of 32, higher embedding sizes after 50 seemed to increase the loss

# Show top 10 movie recommendations to a user

In [ ]:
movie_df = pd.read_csv(movielens_dir / "movies.csv")

# Let us get a user and see the top recommendations.
user_id = df.userId.sample(1).iloc[0]
for user_id in df.userId:
  movies_watched_by_user = df[df.userId == user_id]
  movies_not_watched = movie_df[
      ~movie_df["movieId"].isin(movies_watched_by_user.movieId.values)
  ]["movieId"]
  movies_not_watched = list(
      set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
  )
  movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
  user_encoder = user2user_encoded.get(user_id)
  user_movie_array = np.hstack(
      ([[user_encoder]] * len(movies_not_watched), movies_not_watched)
  )
  ratings = model.predict(user_movie_array).flatten()
  top_ratings_indices = ratings.argsort()[-10:][::-1]
  recommended_movie_ids = [
      movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
  ]

  print("Showing recommendations for user: {}".format(user_id))
  print("====" * 9)
  print("Movies with high ratings from user")
  print("----" * 8)
  top_movies_user = (
      movies_watched_by_user.sort_values(by="rating", ascending=False)
      .head(5)
      .movieId.values
  )
  movie_df_rows = movie_df[movie_df["movieId"].isin(top_movies_user)]
  for row in movie_df_rows.itertuples():
      print(row.title, ":", row.genres)

  print("----" * 8)
  print("Top 10 movie recommendations")
  print("----" * 8)
  recommended_movies = movie_df[movie_df["movieId"].isin(recommended_movie_ids)]
  for row in recommended_movies.itertuples():
      print(row.title, ":", row.genres)

In [39]:
recommendations_data = []

def get_top_recommendations(user_id):
    movies_watched_by_user = df[df.userId == user_id]
    movies_not_watched = movie_df[~movie_df["movieId"].isin(movies_watched_by_user.movieId.values)]["movieId"]
    movies_not_watched = list(set(movies_not_watched).intersection(set(movie2movie_encoded.keys())))
    movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
    user_encoder = user2user_encoded.get(user_id)
    user_movie_array = np.hstack(([[user_encoder]] * len(movies_not_watched), movies_not_watched))
    ratings = model.predict(user_movie_array).flatten()
    top_ratings_indices = ratings.argsort()[-10:][::-1]
    recommended_movie_ids = [movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices]
    return recommended_movie_ids

for user_id in df.userId.unique():
    top_movies_user = movies_watched_by_user.sort_values(by="rating", ascending=False).head(5).movieId.values
    movie_df_rows = movie_df[movie_df["movieId"].isin(top_movies_user)]
    for row in movie_df_rows.itertuples():
        recommendations_data.append({'user_id': user_id, 'movie_title': row.title, 'movie_genre': row.genres})

    recommended_movie_ids = get_top_recommendations(user_id)
    recommended_movies = movie_df[movie_df["movieId"].isin(recommended_movie_ids)]
    for row in recommended_movies.itertuples():
        recommendations_data.append({'user_id': user_id, 'movie_title': row.title, 'movie_genre': row.genres})

recommendations_df = pd.DataFrame(recommendations_data)

recommendations_df.to_excel("user_movie_recommendations.xlsx", index=False)

303/303 [==============================] - 0s 2ms/step
